### To learn tensorflow

- Build Deep NN
- Saver
- Tensorboard
- dropout

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
lr = 1e-3
dropout_prob = 0.08
loadModel = False

In [5]:
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
dropoutProb = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    L1 = tf.nn.dropout(L1, dropoutProb)
    
    tf.summary.histogram("X", X)
    tf.summary.histogram("Weights", W1)
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    L2 = tf.nn.dropout(L2, dropoutProb)
    
    tf.summary.histogram("Weights", W2)

with tf.name_scope('layer3'):
    W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01), name='W3')
    output = tf.matmul(L2, W3)
    
    tf.summary.histogram("Weights", W3)
    tf.summary.histogram("Output", output)

with tf.name_scope('optimizer'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=Y))
    optimizer = tf.train.AdamOptimizer(lr).minimize(loss, global_step=global_step)
    
    # to collect the data I want to see
    tf.summary.scalar('loss', loss)
    

In [6]:
sess = tf.Session()

# saver ===============================================
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path) and loadModel:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print(">>> Loaded the model .......")
else:
    sess.run(tf.global_variables_initializer())

# tensorboard =========================================
'''
on webserver,   tensorboard --logdir=./logs  
and then, we can see it on      http://localhost:6006
'''
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

# start training ======================================
batch_size = 1000
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(30):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        summary, _, cost, _ = sess.run([merged, optimizer, loss, global_step], \
                          {X: batch_xs, Y: batch_ys, dropoutProb: dropout_prob})
        total_cost += cost
        
#         summary = sess.run(merged, {X: batch_xs, Y: batch_ys, dropoutProb: dropout_prob})
        writer.add_summary(summary, global_step=sess.run(global_step))
        
    print('Epoch: {:.3f}  |   avg. cost= {:.3f}'.format(epoch + 1, total_cost/total_batch))

saver.save(sess, './model/mnistModel.ckpt', global_step=global_step)
print(">>> Complete and saved the model !!!")

Epoch: 1.000  |   avg. cost= 3220346532214.087
Epoch: 2.000  |   avg. cost= 17113918857.117
Epoch: 3.000  |   avg. cost= 36126442814.602
Epoch: 4.000  |   avg. cost= 1122550.552
Epoch: 5.000  |   avg. cost= 3632574093.897
Epoch: 6.000  |   avg. cost= 2.303
Epoch: 7.000  |   avg. cost= 2.303
Epoch: 8.000  |   avg. cost= 2.303
Epoch: 9.000  |   avg. cost= 2.303
Epoch: 10.000  |   avg. cost= 14363229907.673
Epoch: 11.000  |   avg. cost= 2.303
Epoch: 12.000  |   avg. cost= 2.303
Epoch: 13.000  |   avg. cost= 1050967544.952
Epoch: 14.000  |   avg. cost= 2.303
Epoch: 15.000  |   avg. cost= 649676913.970
Epoch: 16.000  |   avg. cost= 2.303
Epoch: 17.000  |   avg. cost= 2.303
Epoch: 18.000  |   avg. cost= 2.303
Epoch: 19.000  |   avg. cost= 2.303
Epoch: 20.000  |   avg. cost= 2.303
Epoch: 21.000  |   avg. cost= 2.303
Epoch: 22.000  |   avg. cost= 2.303
Epoch: 23.000  |   avg. cost= 2.303
Epoch: 24.000  |   avg. cost= 8163920209.388
Epoch: 25.000  |   avg. cost= 2.303
Epoch: 26.000  |   avg. co

In [ ]:
is_correct = tf.equal(tf.argmax(output, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images,
                                   Y: mnist.test.labels,
                                   dropoutProb: 1}))

In [ ]:
labels = sess.run(output,
                  feed_dict={X: mnist.test.images,
                             Y: mnist.test.labels,
                             dropoutProb: 1})

fig = plt.figure()
for i in range(10):
    subplot = fig.add_subplot(2, 5, i + 1)
    subplot.set_xticks([])
    subplot.set_yticks([])
    subplot.set_title('%d' % np.argmax(labels[i]))
    subplot.imshow(mnist.test.images[i].reshape((28, 28)),
                   cmap=plt.cm.gray_r)

plt.savefig('./figs/mnist_results.png')
plt.show()